# Градиентный бустинг. Имплементации

In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
data = pd.read_csv("flights.csv")
data = data.sample(frac=0.1, random_state=10)

## LightGBM боевого образца
(пишем вместе)

In [ ]:
'''Your code here.'''


# Стэкинг моделей 

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

def rmse(y_true, y_pred):
    error = (y_true - y_pred) ** 2
    return np.sqrt(np.mean(error))

### Задача 4

- Напишите функцию rmse, рассчитывающую rmse для прогнозных и фактических значениях

In [ ]:
'''Your code here.'''


### Задача 5

- Обучите CatBoostRegressor
- Сохраните прогнозы для тестовой выборки в переменной y_pred_cbm
- Сохраните прогнозы для тренировочной выборки в переменной y_train_pred_cbm

In [ ]:
from catboost import CatBoostRegressor

cbm = CatBoostRegressor(iterations=100, max_depth=4, learning_rate=0.01, loss_function='RMSE', logging_level='Silent')


'''Your code here.'''


print("Train RMSE GB = %.4f" % rmse(y_train, y_train_pred_cbm))
print("Test RMSE GB = %.4f" % rmse(y_test, y_pred_cbm))

### Задача 6

- Отшкалируйте данные
- Обучите линейную регрессию
- Сохраните прогнозы для тестовой выборки в переменной y_pred_lr
- Сохраните прогнозы для тренировочной выборки в переменной y_train_pred_lr

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

'''Your code here.
'''

print("Train RMSE LR = %.4f" % rmse(y_train, y_train_pred_lr))
print("Test RMSE LR = %.4f" % rmse(y_test, y_pred_lr))

Для простоты будем считать, что новый алгоритм $a(x)$ представим как
$$
    a(x)
    =
    \sum_{n = 1}^{N}
    w_n b_n(x),
$$
где $\sum_{n} w_n =1$

In [ ]:
def select_weights(y_true, y_pred_1, y_pred_2):
    grid = np.linspace(0, 1, 1000)
    metric = []
    for w_0 in grid:
        w_1 = 1 - w_0
        y_a = w_0 * y_pred_1 + w_1 * y_pred_2
        metric.append([rmse(y_true, y_a), w_0, w_1])
    return metric

In [ ]:
rmse_blending_train, w_0, w_1 = min(select_weights(y_train, y_train_pred_cbm, y_train_pred_lr), key=lambda x: x[0])

In [ ]:
rmse_blending_train

In [ ]:
rmse(y_test, y_pred_cbm * w_0 +  y_pred_lr * w_1)

В итоге получаем качество на тестовой выборке лучше, чем у каждого алгоритма в отдельности.

Полезные ссылки:


* [Документация XGBoost](https://xgboost.readthedocs.io/en/latest/python/python_intro.html)
* [Документация LightGBM](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html)
* [Документация CatBoost](https://catboost.ai/docs/concepts/python-quickstart.html)
* [Видео про стекинг](https://www.coursera.org/lecture/competitive-data-science/stacking-Qdtt6)
* [Продвинутый вариант стекинга с использованием нейросетей](https://www.coursera.org/learn/competitive-data-science/lecture/s8RLi/stacknet)

# PCA и кластеризация

In [7]:
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

### Задача 7

- Объявите PCA с двумя компонентами. Параметр - n_components. 
- Методом fit обучите на данных ирисов PCA
- В поле explained_variance_ratio_ указаны, какую долю дисперсии смогла объяснить каждая из полученных компонент. Сколько процентов дисперсии мы потеряли преобразованием?
- Методом transform примените преобразование к исходным данным.
- Визуализируйте данные в виде точечной диаграммы и окрасьте точки в цвета соответствующего класса цветков.